<a href="https://colab.research.google.com/github/psa2001/computer_vision/blob/main/tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd

/root


In [ ]:
cd ..

/


In [ ]:
ls

bin/      dev/   lib32/  opt/   sbin/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  proc/  srv/    tmp/
content/  home/  media/  root/  swift/  tools/
datalab/  lib/   mnt/    run/   sys/    usr/


In [ ]:
import os
from pathlib import Path
from IPython.display import clear_output


In [ ]:
os.chdir('content/drive/My Drive/car_tracking')

In [ ]:
!chmod +x run.py

In [ ]:
ls

additonal_files.zip   metrics.py             ssd_utils.py
build.sh              public_tests.zip       ssd_v2.py
config.py             __pycache__/           tests/
cross_correlation.py  run.py*                tracker.py
description.pdf       solution_template.zip  utils.py
detection.py          ssd_layers.py          weights_SSD300.hdf5


In [ ]:
import numpy as np
import os

from PIL import Image
from skimage.transform import resize
from skimage import io
from tensorflow.keras.applications.imagenet_utils import preprocess_input

from config import VOC_CLASSES, bbox_util, model
from utils import get_color


def detection_cast(detections):
    """Helper to cast any array to detections numpy array.
    Even empty.
    """
    return np.array(detections, dtype=np.int32).reshape((-1, 5))


def rectangle(shape, ll, rr, line_width=5):
    """Draw rectangle on numpy array.

    rr, cc = rectangle(frame.shape, (ymin, xmin), (ymax, xmax))
    frame[rr, cc] = [0, 255, 0] # Draw green bbox
    """
    ll = np.minimum(np.array(shape[:2], dtype=np.int32) - 1, np.maximum(ll, 0))
    rr = np.minimum(np.array(shape[:2], dtype=np.int32) - 1, np.maximum(rr, 0))
    result = []

    for c in range(line_width):
        for i in range(ll[0] + c, rr[0] - c + 1):
            result.append((i, ll[1] + c))
            result.append((i, rr[1] - c))
        for j in range(ll[1] + c + 1, rr[1] - c):
            result.append((ll[0] + c, j))
            result.append((rr[0] - c, j))

    return tuple(zip(*result))


def extract_detections(frame, min_confidence=0.6, labels=None):
    """Extract detections from frame.

    frame: numpy array WxHx3
    returns: numpy int array Cx5 [[label_id, xmin, ymin, xmax, ymax]]
    """
    # Write code here
    # First, convert frame to float and resize to 300x300
    frame = np.array(frame, dtype = np.float64)
    print(frame.shape)
    x_shape_coef = frame.shape[1] 
    y_shape_coef = frame.shape[0]
    frame_shaped = resize(frame, (300, 300))
    # Then use preprocess_input, model.predict and bbox_util.detection_out
    # Use help(...) function to help
    frame_with_batches = np.expand_dims(frame_shaped, axis = 0)
    preprocess_frame = preprocess_input(frame_with_batches)
    results = model.predict(preprocess_frame)
    results = bbox_util.detection_out(results)

    # Select detections with confidence > min_confidence
    results = np.array(results)
    results = results[0,:]
    a = (results[:, 1] > min_confidence)
    results = results[a, :]
    # If label set is known, use it
    if labels is not None:
        result_labels = results[:, 0].astype(np.int32)
        indeces = [i for i, l in enumerate(result_labels) if VOC_CLASSES[l - 1] in labels]
        results = results[indeces]

    # Remove confidence column from result
    a[:, 1] = -1
    a = a[a != -1].reshape((a.shape[0], -1))
    results = a

    # Resize detection coords to input image shape.
    # Didn't you forget to save it before resize?
    results[:, [1, 3]] *= x_shape_coef
    results[:, [2, 4]] *= y_shape_coef

    # Return result
    return detection_cast(results)


def draw_detections(frame, detections):
    """Draw detections on frame.

    Hint: help(rectangle) would help you.
    Use get_color(label) to select color for detection.
    """
    frame = frame.copy()

    # Write code here
    rectangling = rectangle(frame.shape, detections[1:3], detections[3:])
    color_rectangle = get_color(rectangling)

    return frame


def main():
    dirname = os.path.dirname(__file__)
    frame = Image.open(os.path.join(dirname, 'data', 'test.png'))
    frame = np.array(frame)
    print('lets_start_resize')

    detections = extract_detections(frame)
    frame = draw_detections(frame, detections)

    io.imshow(frame)
    io.show()


if __name__ == '__main__':
    main()

In [ ]:
!chmod +x run.py

In [ ]:
!./run.py unittest detection

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/My Drive/car_tracking, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

tests/00_unittest_detection_input/test.py::test_detection PASSED         [100%]

=========================== 1 passed in 7.75 seconds ===========================


In [ ]:
def iou_score(bbox1, bbox2):
    """Jaccard index or Intersection over Union.

    https://en.wikipedia.org/wiki/Jaccard_index

    bbox: [xmin, ymin, xmax, ymax]
    """

    assert len(bbox1) == 4
    assert len(bbox2) == 4

    # Write code here
    #sort на всякий случай)
    x_first = np.sort(np.array([bbox1[::2]]))[0]
    y_first = np.sort(np.array([bbox1[1::2]]))[0]
    x_second = np.sort(np.array([bbox2[::2]]))[0]
    y_second = np.sort(np.array([bbox2[1::2]]))[0]
    dice_score = 0.0
    if(x_first[1] < x_second[0] or x_first[0] > x_second[1]):
      dice_score = 0.0
    elif((y_first[1] < y_second[0] or y_first[0] > y_second[1])):
      dice_score = 0.0
    else:
      x_all = np.sort((np.concatenate((x_first, x_second))).reshape((-1)))
      y_all = np.sort(np.concatenate((y_first, y_second)).reshape((-1)))
      chis = np.abs((x_all[2] - x_all[1]) * (y_all[2] - y_all[1]))
      znam = -chis + np.abs((y_first[1] - y_first[0]) * (x_first[1] - x_first[0])) + np.abs((y_second[1] - y_second[0]) * (x_second[0] - x_second[1]))
      dice_score = chis / znam

    return dice_score

In [ ]:
def test_iou():
        base_bbox = [16, 25, 42, 80]
        bboxes = [
            [23, 36, 40, 71],
            [2, 8, 12, 100],
            [10, 27, 44, 82],
            [14, 6, 78, 42],
            [18, 22, 44, 80]
        ]
    
        gt_metrics = [0.416, 0, 0.717, 0.134, 0.816]
        result_metrics = [iou_score(base_bbox, bbox) for bbox in bboxes]
        print(result_metrics)

In [ ]:
test_iou()

x_all [16 23 40 42]
595 1430
x_all [10 16 42 44]
1378 1922
x_all [14 16 42 78]
442 3292
x_all [16 18 42 44]
1320 1618
[0.4160839160839161, 0.0, 0.7169614984391259, 0.13426488456865127, 0.8158220024721878]


In [ ]:
!./run.py unittest iou

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/My Drive/car_tracking, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

tests/01_unittest_iou_input/test.py::test_iou PASSED                     [100%]

=========================== 1 passed in 1.12 seconds ===========================


In [ ]:
class Tracker:
    """Generate detections and build tracklets."""
    def __init__(self, return_images=True, lookup_tail_size=80, labels=None):
        self.return_images = return_images  # Return images or detections?
        
        self.frame_index = 0 #current frame_number
        
        self.labels = labels  # Tracker label list
        self.detection_history = []  # Saved detection list

        self.last_detected = {} #every id correlate last frame, where it was detected
        
        self.tracklet_count = 0  # Counter to enumerate tracklets

        # We will search tracklet at last lookup_tail_size frames
        self.lookup_tail_size = lookup_tail_size

    def new_label(self):
        """Get new unique label."""
        self.tracklet_count += 1
        return self.tracklet_count - 1

    def init_tracklet(self, frame):
        """Get new unique label for every detection at frame and return it."""
        # Write code here
        # Use extract_detections and new_label
        label = extract_detections(frame)
        for i in range(label.shape[0]):
          new_label()
        return label

    @property
    def prev_detections(self):
        """Get detections at last lookup_tail_size frames from detection_history.

        One detection at one id.
        """
        detections = []
        # Write code here

        for i in (self.last_detected.keys()):
          # i - id; cur - номер
          cur = self.last_detected[i]
          if(self.frame_index - self.lookup_tail_size <= cur):
            a = detection_history[cur]
            detections.append(a[a[0] == i])

        detections = np.array(detections)
        return detection_cast(detections)

    def bind_tracklet(self, detections):
        """Set id at first detection column.

        Find best fit between detections and previous detections.

        detections: numpy int array Cx5 [[label_id, xmin, ymin, xmax, ymax]]
        return: binded detections numpy int array Cx5 [[tracklet_id, xmin, ymin, xmax, ymax]]
        """
        detections = detections.copy()
        prev_detections = self.prev_detections

        # Write code here
        previous = prev_detections()
        # Step 1: calc pairwise detection IOU
        iou_matrix = np.zeros((previous.shape[0], detections.shape[0]))

        for i in range(previous.shape[0]):
          for j in range(detections.shape[0]):
            cur_iou = iou_score(previous[1:], detections[1:])
            iou[i][j] = cur_iou
        # Step 2: sort IOU list
        argmax_arr = np.argmax(iou, axis = 1)
        # Step 3: fill detections[:, 0] with best match
        # One matching for each id
        for i in range(np.size(argmax_arr)):
          min_iou = 0
          if(iou[i][argmax[i]] > min_iou):
            previous[i][0] = detections[i][0]
          else:
            new_label()
            unique_label = np.max(self.tracklet_count.keys()) + 1
            previous[i][0] = unique_label

        # Step 4: assign new tracklet id to unmatched detections
        detections = previous
        return detection_cast(detections)

    def save_detections(self, detections):
        """Save last detection frame number for each label."""
        for label in detections[:, 0]:
            self.last_detected[label] = self.frame_index

    def update_frame(self, frame):
        if not self.frame_index:
            # First frame should be processed with init_tracklet function
            detections = self.init_tracklet(frame)
        else:
            # Every Nth frame should be processed with CNN (very slow)
            # First, we extract detections
            detections = extract_detections(frame, labels=self.labels)
            # Then bind them with previous frames
            # Replacing label id to tracker id is performing in bind_tracklet function
            detections = self.bind_tracklet(detections)

        # After call CNN we save frame number for each detection
        self.save_detections(detections)
        # Save detections and frame to the history, increase frame counter
        self.detection_history.append(detections)
        self.frame_index += 1

        # Return image or raw detections
        # Image usefull to visualizing, raw detections to metric
        if self.return_images:
            return draw_detections(frame, detections)
        else:
            return detections

In [ ]:
!./run.py unittest tracker

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/My Drive/car_tracking, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

tests/02_unittest_tracker_input/test.py::test_tracker PASSED             [100%]

========================== 1 passed in 11.78 seconds ===========================


In [ ]:
def iou_score(bbox1, bbox2):
    """Jaccard index or Intersection over Union.

    https://en.wikipedia.org/wiki/Jaccard_index

    bbox: [xmin, ymin, xmax, ymax]
    """

    assert len(bbox1) == 4
    assert len(bbox2) == 4

    # Write code here
    #sort на всякий случай)
    
    import numpy as np
    
    x_first = np.sort(np.array([bbox1[::2]]))[0]
    y_first = np.sort(np.array([bbox1[1::2]]))[0]
    x_second = np.sort(np.array([bbox2[::2]]))[0]
    y_second = np.sort(np.array([bbox2[1::2]]))[0]
    dice_score = 0.0
    if(x_first[1] < x_second[0] or x_first[0] > x_second[1]):
      dice_score = 0.0
    elif((y_first[1] < y_second[0] or y_first[0] > y_second[1])):
      dice_score = 0.0
    else:
      x_all = np.sort((np.concatenate((x_first, x_second))).reshape((-1)))
      y_all = np.sort(np.concatenate((y_first, y_second)).reshape((-1)))
      chis = np.abs((x_all[2] - x_all[1]) * (y_all[2] - y_all[1]))
      znam = -chis + np.abs((y_first[1] - y_first[0]) * (x_first[1] - x_first[0])) + np.abs((y_second[1] - y_second[0]) * (x_second[0] - x_second[1]))
      dice_score = chis / znam

    return dice_score


In [ ]:
def motp(obj, hyp, threshold=0.5):
    """Calculate MOTP

    obj: list
        Ground truth frame detections.
        detections: numpy int array Cx5 [[id, xmin, ymin, xmax, ymax]]

    hyp: list
        Hypothetical frame detections.
        detections: numpy int array Cx5 [[id, xmin, ymin, xmax, ymax]]

    threshold: IOU threshold
    """

    dist_sum = 0  # a sum of IOU distances between matched objects and hypotheses
    match_count = 0

    matches = {}  # matches between object IDs and hypothesis IDs

    # For every frame
    for frame_obj, frame_hyp in zip(obj, hyp):
        # Write code here
        frame_obj = np.array(frame_obj) 
        frame_hyp = np.array(frame_hyp) 

        # Step 1: Convert frame detections to dict with IDs as keys
        
        #not sure
        for i in range(frame_obj.shape[0]):
          cur_id_frame = frame_obj[i, 0]
          cur_id_hyp = frame_hyp[frame_hyp[:, 0] == frame_obj[i, 0]]
          if(np.size(cur_id_hyp) != 0):
            iou_cur = iou_score(frame_obj[i, 1:], cur_id_hyp[0,1:])
            if(iou_cur > threshold):
              match_count += 1
              dist_sum += iou_cur
        matches = {}

        # Step 2: Iterate over all previous matches
        # If object is still visible, hypothesis still exists
        # and IOU distance > threshold - we've got a match
        # Update the sum of IoU distances and match count
        # Delete matched detections from frame detections

        

        # Step 3: Calculate pairwise detection IOU between remaining frame detections
        # Save IDs with IOU > threshold

        # Step 4: Iterate over sorted pairwise IOU
        # Update the sum of IoU distances and match count
        # Delete matched detections from frame detections

        # Step 5: Update matches with current matched IDs
        pass

    # Step 6: Calculate MOTP
    MOTP = dist_sum / match_count

    return MOTP

In [ ]:
def test_motp():
        gt_tracks = [
            [[0, 0, 0, 150, 100], [1, 400, 0, 500, 150]],
            [[0, 50, 50, 200, 150], [1, 400, 100, 500, 250]],
            [[0, 150, 100, 300, 200], [1, 350, 150, 450, 300]]
        ]
        result_tracks = [
            [[0, 15, 15, 145, 115], [1, 380, 20, 510, 150]],
            [[0, 40, 40, 190, 160], [1, 400, 140, 520, 270]],
            [[0, 150, 150, 280, 240], [1, 330, 170, 450, 290]]
        ]
    
        gt_metric = 0.665
        result_metric = motp(gt_tracks, result_tracks)
        print(result_metric)
      
test_motp()

0.6654939718866701


In [ ]:
!./run.py unittest motp

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/My Drive/car_tracking, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

tests/03_unittest_motp_input/test.py::test_motp PASSED                   [100%]

=========================== 1 passed in 0.17 seconds ===========================


In [ ]:
!./run.py unittest baseline

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/My Drive/car_tracking, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

tests/04_unittest_baseline_input/test.py::test_baseline PASSED           [100%]

========================== 1 passed in 131.24 seconds ==========================


(1, 4)

In [ ]:
a = np.random.randint(0,100,12).reshape((3,4))
a

array([[13, 15, 34, 44],
       [21, 31, 49, 76],
       [85, 11, 95, 59]])

In [ ]:
np.argmax(a, axis = 0)

array([2, 1, 2, 1])

In [ ]:
import numpy as np
import os

from moviepy.editor import VideoFileClip
from skimage.feature import match_template
from skimage.color import rgb2gray

from detection import extract_detections, draw_detections, detection_cast
from tracker import Tracker


def gaussian(shape, x, y, dx, dy):
    """Return gaussian for tracking.

    shape: [width, height]
    x, y: gaussian center
    dx, dy: std by x and y axes

    return: numpy array (width x height) with gauss function, center (x, y) and std (dx, dy)
    """
    Y, X = np.mgrid[0:shape[0], 0:shape[1]]
    return np.exp(-(X - x) ** 2 / dx ** 2 - (Y - y) ** 2 / dy ** 2)


class CorrelationTracker(Tracker):
    """Generate detections and building tracklets."""
    def __init__(self, detection_rate=5, **kwargs):
        super().__init__(**kwargs)
        self.detection_rate = detection_rate  # Detection rate
        self.prev_frame = None  # Previous frame (used in cross correlation algorithm)

    def build_tracklet(self, frame):
        """Between CNN execution uses normalized cross-correlation algorithm (match_template)."""
        detections = []
        # Write code here
        import numpy as np
        # Apply rgb2gray to frame and previous frame
        gray_frame = rgb2gray(frame)
        prev_fray_scale = rgb2gray(self.prev_frame)

        # For every previous detection
        # Use match_template + gaussian to extract detection on current frame
        for label, xmin, ymin, xmax, ymax in self.detection_history[-1]:
            # Step 0: Extract prev_bbox from prev_frame
            prev_bbox = self.prev_frame[xmin:xmax, ymin:ymax]
            # Step 1: Extract new_bbox from current frame with the same coordinates
            new_bbox = frame[xmin:xmax, ymin:ymax]
            # Step 2: Calc match_template between previous and new bbox
            # Use padding
            matching = match_template(prev_bbox, new_bbox, pad_input = True)
            # Step 3: Then multiply matching by gauss function
            # Find argmax(matching * gauss)
            xyz = np.argmax(matching * gaussian(label.shape[0], x_min, y_min, x_max - x_min, y_max - y_min))
            # Step 4: Append to detection list
            detections.append(xyz)

        return detection_cast(detections)

    def update_frame(self, frame):
        if not self.frame_index:
            detections = self.init_tracklet(frame)
            self.save_detections(detections)
        elif self.frame_index % self.detection_rate == 0:
            detections = extract_detections(frame, labels=self.labels)
            detections = self.bind_tracklet(detections)
            self.save_detections(detections)
        else:
            detections = self.build_tracklet(frame)

        self.detection_history.append(detections)
        self.prev_frame = frame
        self.frame_index += 1

        if self.return_images:
            return draw_detections(frame, detections)
        else:
            return detections


def main():
    dirname = os.path.dirname(__file__)
    input_clip = VideoFileClip(os.path.join(dirname, 'data', 'test.mp4'))

    tracker = CorrelationTracker()
    input_clip.fl_image(tracker.update_frame).preview()


if __name__ == '__main__':
    main()



In [ ]:
def motp_mota(obj, hyp, threshold=0.5):
    """Calculate MOTP/MOTA

    obj: list
        Ground truth frame detections.
        detections: numpy int array Cx5 [[id, xmin, ymin, xmax, ymax]]

    hyp: list
        Hypothetical frame detections.
        detections: numpy int array Cx5 [[id, xmin, ymin, xmax, ymax]]

    threshold: IOU threshold
    """

    dist_sum = 0  # a sum of IOU distances between matched objects and hypotheses
    match_count = 0
    missed_count = 0
    false_positive = 0
    mismatch_error = 0

    matches = {}  # matches between object IDs and hypothesis IDs

    # For every frame
    #not_sure
    all_number_of_object = 0
    for frame_obj, frame_hyp in zip(obj, hyp):
        all_number_of_object += 2
        # Step 1: Convert frame detections to dict with IDs as keys
        frame_obj = np.array(frame_obj) 
        frame_hyp = np.array(frame_hyp) 

        # Step 1: Convert frame detections to dict with IDs as keys
        
        matched = 0

        for i in range(frame_obj.shape[0]):
          cur_id_frame = frame_obj[i, 0]
          cur_id_hyp = frame_hyp[frame_hyp[:, 0] == frame_obj[i, 0]]
          if(np.size(cur_id_hyp) != 0):
            matched += 1
            iou_cur = iou_score(frame_obj[i, 1:], cur_id_hyp[0,1:])
            if(iou_cur > threshold):
              match_count += 1
              dist_sum += iou_cur
            else:
              mismatch_error += 1
        
        false_positive += frame_hyp.shape[0] - matched
        
        missed_count += frame_obj.shape[0] - matched
        matched = 0

        print(match_count, missed_count, false_positive, mismatch_error)
        # Step 2: Iterate over all previous matches
        # If object is still visible, hypothesis still exists
        # and IOU distance > threshold - we've got a match
        # Update the sum of IoU distances and match count
        # Delete matched detections from frame detections

        # Step 3: Calculate pairwise detection IOU between remaining frame detections
        # Save IDs with IOU > threshold

        # Step 4: Iterate over sorted pairwise IOU
        # Update the sum of IoU distances and match count
        # Delete matched detections from frame detections

        # Step 5: If matched IDs contradict previous matched IDs - increase mismatch error

        # Step 6: Update matches with current matched IDs

        # Step 7: Errors
        # All remaining hypotheses are considered false positives
        # All remaining objects are considered misses
        pass

    # Step 8: Calculate MOTP and MOTA
    MOTP = dist_sum / match_count
    match_count + missed_count + false_positive  + mismatch_error 
    MOTA = 1 - (missed_count + false_positive  + mismatch_error)/ (all_number_of_object)

    return [MOTP, MOTA]


In [ ]:
def test_motp_mota():
        gt_tracks = [
            [[0, 0, 0, 150, 100], [1, 400, 0, 500, 150]],
            [[0, 50, 50, 200, 150], [1, 400, 100, 500, 250]],
            [[0, 150, 100, 300, 200], [1, 350, 150, 450, 300]]
        ]
        result_tracks = [
            [[0, 15, 15, 145, 115], [1, 380, 20, 510, 150]],
            [[0, 40, 40, 190, 160], [1, 400, 140, 520, 270]],
            [[0, 150, 150, 280, 240], [2, 330, 170, 450, 290]]
        ]
    
        gt_metrics = np.array([0.665, 0.5])
        result_metrics = motp_mota(gt_tracks, result_tracks)
        print(result_metrics)

test_motp_mota()

2 0 0 0
4 0 0 0
4 1 1 1
[0.6594536717548893, 0.5]


In [ ]:
!./run.py unittest motp_mota

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/My Drive/car_tracking, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

tests/05_unittest_motp_mota_input/test.py::test_motp_mota PASSED         [100%]

=========================== 1 passed in 1.56 seconds ===========================


In [ ]:
!./run.py unittest tracking

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/My Drive/car_tracking, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

tests/06_unittest_tracking_input/test.py::test_tracking FAILED           [100%]

=================================== FAILURES ===================================
________________________________ test_tracking _________________________________

    def test_tracking():
        data_dir = join(dirname(__file__), 'data')
        video_names = ['road', 'jogging', 'street']
    
        gt_data = np.array([
            [0.936, 0.968],
            [0.676, 0.697],
            [0.694, 0.328]
        ])
    
        threshold = 0.05
    
        for video_name, gt_metrics in zip(video_names, gt_data):
            input_clip = VideoFileClip(join(

In [ ]:
    def bind_tracklet(self, detections):
        """Set id at first detection column.

        Find best fit between detections and previous detections.

        detections: numpy int array Cx5 [[label_id, xmin, ymin, xmax, ymax]]
        return: binded detections numpy int array Cx5 [[tracklet_id, xmin, ymin, xmax, ymax]]
        """
        detections = detections.copy()
        prev_detections = self.prev_detections
        # Write code here
        previous = prev_detections
 
        # Step 1: calc pairwise detection IOU
        iou = np.zeros((previous.shape[0], detections.shape[0]))

        for i in range(previous.shape[0]):
          for j in range(detections.shape[0]):
            cur_iou = iou_score(previous[i, 1:], detections[j, 1:])
            iou[i][j] = cur_iou
        # Step 2: sort IOU list
        if(np.size(iou)!= 0):
        	argmax_arr = np.argmax(iou, axis = 0)
            
          # Step 3: fill detections[:, 0] with best match
          # One matching for each id
          for i in range(np.size(argmax_arr)):
            min_iou = 0
            if(iou[argmax_arr[i]][i] > min_iou):
              detections[i][0] = previous[i][0]
            else:
              unique_label = self.new_label()
              print(unique_label, detections.shape)
              detections[i,0] = unique_label
        else:
          for i in range(detections.shape[0]):
              unique_label = self.new_label()
              detections[i,0] = unique_label
        # Step 4: assign new tracklet id to unmatched detections
        return detection_cast(detections)

In [ ]:
import numpy as np
import os

from moviepy.editor import VideoFileClip
from skimage.feature import match_template
from skimage.color import rgb2gray

from detection import extract_detections, draw_detections, detection_cast
from tracker import Tracker


def gaussian(shape, x, y, dx, dy):
    """Return gaussian for tracking.

    shape: [width, height]
    x, y: gaussian center
    dx, dy: std by x and y axes

    return: numpy array (width x height) with gauss function, center (x, y) and std (dx, dy)
    """
    print(shape)
    Y, X = np.mgrid[0:shape[0], 0:shape[1]]
    return np.exp(-(X - x) ** 2 / dx ** 2 - (Y - y) ** 2 / dy ** 2)


class CorrelationTracker(Tracker):
    """Generate detections and building tracklets."""
    def __init__(self, detection_rate=5, **kwargs):
        super().__init__(**kwargs)
        self.detection_rate = detection_rate  # Detection rate
        self.prev_frame = None  # Previous frame (used in cross correlation algorithm)

    def build_tracklet(self, frame):
        """Between CNN execution uses normalized cross-correlation algorithm (match_template)."""
        detections = []
        # Write code here
        import numpy as np
        # Apply rgb2gray to frame and previous frame
        gray_frame = rgb2gray(frame)
        prev_fray_scale = rgb2gray(self.prev_frame)

        # For every previous detection
        # Use match_template + gaussian to extract detection on current frame
        for label, xmin, ymin, xmax, ymax in self.detection_history[-1]:
            # Step 0: Extract prev_bbox from prev_frame
            prev_bbox = self.prev_frame[ymin:ymax ,xmin:xmax]
            # Step 1: Extract new_bbox from current frame with the same coordinates
            new_bbox = frame[ymin:ymax ,xmin:xmax]
            # Step 2: Calc match_template between previous and new bbox
            # Use padding
            matching = match_template( new_bbox,prev_bbox,pad_input=True)
            # Step 3: Then multiply matching by gauss function
            # Find argmax(matching * gauss)
            print(matching.shape)
            #not sure
            gaus = gaussian(new_bbox.shape, new_bbox.shape[1] // 2, new_bbox.shape[0] // 2, prev_bbox.shape[0], prev_bbox.shape[1])
            xyz = matching * gaus
            ij = np.unravel_index(np.argmax(xyz), xyz.shape)
            x, y = ij[::-1]
            # Step 4: Append to detection list
            
            x1 = np.min(np.max(xmin + x - new_bbox.shape[1] // 2, 0) ,label.shape[1] - 1)
            x2 = np.min(np.max(xmax + x - new_bbox.shape[1] // 2, 0) ,label.shape[0] - 1)
            y1 = np.min(np.max(ymin + y - new_bbox.shape[0] // 2, 0) ,label.shape[0] - 1)
            y2 = np.min(np.max(ymax + y - new_bbox.shape[0] // 2, 0) ,label.shape[0] - 1)
            detect_cur = np.array([label, x1, y1, x2, y2])
            detections.append(detect_cur)

        return detection_cast(detections)

    def update_frame(self, frame):
        if not self.frame_index:
            detections = self.init_tracklet(frame)
            self.save_detections(detections)
        elif self.frame_index % self.detection_rate == 0:
            detections = extract_detections(frame, labels=self.labels)
            detections = self.bind_tracklet(detections)
            self.save_detections(detections)
        else:
            detections = self.build_tracklet(frame)

        self.detection_history.append(detections)
        self.prev_frame = frame
        self.frame_index += 1

        if self.return_images:
            return draw_detections(frame, detections)
        else:
            return detections


def main():
    dirname = os.path.dirname(__file__)
    input_clip = VideoFileClip(os.path.join(dirname, 'data', 'test.mp4'))

    tracker = CorrelationTracker()
    input_clip.fl_image(tracker.update_frame).preview()


if __name__ == '__main__':
    main()

